<a href="https://www.kaggle.com/code/averma111/pytroch-llm-science-exam?scriptVersionId=140045608" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [22]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset, load_metric
from datasets import Dataset
from transformers import AutoTokenizer
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from transformers import AutoTokenizer, DebertaV2ForMultipleChoice,TrainingArguments, Trainer
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer
from typing import Optional, Union
from tqdm import tqdm
import torch,os
from dataclasses import dataclass
from transformers import AutoTokenizer, DebertaV2Model
from typing import Optional, Union
import numpy as np
from colorama import Fore, Back, Style
from tqdm.notebook import tqdm
import gc
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer, AutoModel, EarlyStoppingCallback
from sklearn.model_selection import KFold
from IPython.display import IFrame

import warnings
warnings.filterwarnings('ignore')
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
 

/kaggle/input/kaggle-llm-science-exam/sample_submission.csv
/kaggle/input/kaggle-llm-science-exam/train.csv
/kaggle/input/kaggle-llm-science-exam/test.csv
/kaggle/input/additional-train-data-for-llm-science-exam/6000_train_examples.csv
/kaggle/input/additional-train-data-for-llm-science-exam/extra_train_set.csv


In [23]:
class DataAcquisition:
    
    def __init__(self):
        pass
    
    def prepare_data(self):
        path = '/kaggle/input/kaggle-llm-science-exam/train.csv'
        df = pd.read_csv(
            path, 
            sep=',', 
            low_memory=False
        )
        
        X = df.copy()
        return X
    
    def prepare_extra_data(self):
        path = '/kaggle/input/additional-train-data-for-llm-science-exam/extra_train_set.csv'
        df = pd.read_csv(
            path, 
            sep=',', 
            low_memory=False
        )
        
        X = df.copy()
        return X
    
    
    def prepare_data_test(self):
        path = '/kaggle/input/kaggle-llm-science-exam/test.csv'
        df = pd.read_csv(
            path, 
            sep=',', 
            low_memory=False
        )
        
        X = df.copy()
        return X
    
    
    def new_features(self,df):
        pass
    
    
    
    def preprocessing_dataset(self,df):
        return  Dataset.from_pandas(df)
        
    def preprocessing_test(self,df):
        for idx in tqdm(range(df.shape[0]) , total = df.shape[0]):
            df["answer"][idx] = df[df["answer"][idx]][idx]
        return df
    
        
acq = DataAcquisition()

train_df = acq.prepare_data()
extra_df = acq.prepare_extra_data()
test_df = acq.prepare_data_test()

In [24]:
train_df.head()

,id,prompt,A,B,C,D,E,answer
0,0,Which of the following statements accurately d...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...,D
1,1,Which of the following is an accurate definiti...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,A
2,2,Which of the following statements accurately d...,The triskeles symbol was reconstructed as a fe...,The triskeles symbol is a representation of th...,The triskeles symbol is a representation of a ...,The triskeles symbol represents three interloc...,The triskeles symbol is a representation of th...,A
3,3,What is the significance of regularization in ...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,C
4,4,Which of the following statements accurately d...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,D


In [25]:
extra_df.head()

,prompt,C,E,D,B,A,answer
0,"In relation to Eunice Fay McKenzie's career, w...",McKenzie gained recognition for her role as a ...,McKenzie's successful career in sound films co...,McKenzie's collaborations with director Blake ...,McKenzie is primarily remembered for her starr...,McKenzie showcased her singing talents in nume...,B
1,How does Modified Newtonian Dynamics (MOND) im...,MOND is a theory that reduces the observed mis...,MOND's impact on the observed missing baryonic...,MOND is a theory that eliminates the observed ...,MOND explains the missing baryonic mass in gal...,MOND is a theory that increases the discrepanc...,E
2,Which of the following statements accurately d...,Ray Montgomerie is a former footballer who pla...,Ray Montgomerie is a former footballer who pla...,Ray Montgomerie is a former footballer who pla...,Ray Montgomerie is a former footballer who pla...,Ray Montgomerie is a former footballer who pla...,B
3,What is the significance of the Museum of the ...,The Museum of the Occupation of Latvia was est...,The Museum of the Occupation of Latvia is a mu...,The Museum of the Occupation of Latvia primari...,The Museum of the Occupation of Latvia showcas...,The Museum of the Occupation of Latvia is a me...,C
4,What was the previous name of the Christian Sc...,The Evangelical School and Chapel for the Deaf...,The Evangelical School for the Blind (ESB),The Evangelical School for the Deaf (ESD),The Christian School for the Blind (CSB),The Christian School for the Deaf (CSD),D


In [26]:
concat_df = pd.concat([
    train_df,
    extra_df
])
train_df.reset_index(inplace=True, drop=True)
concat_df.shape

(700, 8)

In [27]:
train_ds=acq.preprocessing_dataset(concat_df)


In [28]:
def map3(y_true, y_pred):
    m = (y_true.reshape((-1,1)) == y_pred)
    return np.mean(np.where(m.any(axis=1), m.argmax(axis=1)+1, np.inf)**(-1))

In [36]:
# Load model directly
from transformers import AutoModel
model = AutoModel.from_pretrained("microsoft/deberta-v2-xxlarge")
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v2-xxlarge")

Some weights of the model checkpoint at microsoft/deberta-v2-xxlarge were not used when initializing DebertaV2Model: ['lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.dense.bias']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [37]:
options = 'ABCDE'
indices = list(range(5))
option_to_index = {option: index for option, index in zip(options, indices)}
index_to_option = {index: option for option, index in zip(options, indices)}

def preprocess(example):
    first_sentence = [example['prompt']] * 5
    second_sentence = [example[option] for option in options]
    tokenized_example = tokenizer(first_sentence, second_sentence, truncation=True)
    tokenized_example['label'] = option_to_index[example['answer']]
    return tokenized_example


In [38]:
tokenized_train_ds = train_ds.map(preprocess, batched=False, remove_columns=['prompt', 'A', 'B', 'C', 'D', 'E', 'answer'])

  0%|          | 0/700 [00:00<?, ?ex/s]

In [39]:
tokenized_train_ds

Dataset({
    features: ['id', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask', 'label'],
    num_rows: 700
})

In [40]:
@dataclass
class DataCollatorForMultipleChoice:
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    
    def __call__(self, features):
        label_name = "label" if 'label' in features[0].keys() else 'labels'
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]['input_ids'])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])
        
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors='pt',
        )
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch['labels'] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [44]:
%%time

final_dfs = pd.DataFrame()
kf = KFold(n_splits=4, shuffle=True, random_state=42)
cv_list = []
for fold, (idx_tr, idx_va) in enumerate(kf.split(concat_df)):
    
    train_set = concat_df.loc[idx_tr, ['prompt', 'A', 'B', 'C', 'D', 'E', 'answer']]
    valid_set = concat_df.loc[idx_va, ['prompt', 'A', 'B', 'C', 'D', 'E', 'answer']]
    valid_label = concat_df.loc[idx_va, 'answer'].values
    train_set = Dataset.from_pandas(train_set)
    tokenized_train = train_set.map(preprocess, remove_columns=['prompt', 'A', 'B', 'C', 'D', 'E', 'answer'])
    valid_set = Dataset.from_pandas(valid_set)
    tokenized_valid = valid_set.map(preprocess, remove_columns=['prompt', 'A', 'B', 'C', 'D', 'E', 'answer'])
    
    training_args = TrainingArguments(
        output_dir='./',
        overwrite_output_dir=True,
        load_best_model_at_end=True,
        save_total_limit=1,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        warmup_ratio=0.8,
        learning_rate=2e-6,
        per_device_train_batch_size=1,
        per_device_eval_batch_size=2,
        num_train_epochs=6,
        report_to='none',
        seed=42
    )
    model = AutoModelForMultipleChoice.from_pretrained(deberta_v3_large)

    trainer = Trainer(
        model=model,
        args=training_args,
        tokenizer=tokenizer,
        data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer),
        train_dataset=tokenized_train,
        eval_dataset=tokenized_valid,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
    )
    trainer.train()
    
    valid_pred = trainer.predict(tokenized_valid).predictions
    valid_pred_ids = np.argsort(-valid_pred, 1)
    valid_pred_letters = np.array(list('ABCDE'))[valid_pred_ids][:, :3]
    valid_map3 = map3(valid_label, valid_pred_letters)
    print(f"{Fore.RED}{Style.BRIGHT}Fold {fold}: MAP@3 = {valid_map3:.5f}{Style.RESET_ALL}")
    cv_list.append(valid_map3)
    
    test_predictions = trainer.predict(tokenized_test_ds).predictions
    fold_predict_df = pd.DataFrame(test_predictions, columns=[f'{x}{fold}' for x in ['A','B','C','D','E']])
    final_dfs = pd.concat([final_dfs, fold_predict_df], axis=1)
    
    del model, trainer, tokenized_train, tokenized_valid, train_set, valid_set
    gc.collect()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /opt/conda/lib/python3.10/site-packages/IPython/core/magics/execution.py:1325 in time            │
│                                                                                                  │
│   1322 │   │   else:                                                                             │
│   1323 │   │   │   st = clock2()                                                                 │
│   1324 │   │   │   try:                                                                          │
│ ❱ 1325 │   │   │   │   exec(code, glob, local_ns)                                                │
│   1326 │   │   │   │   out=None                                                                  │
│   1327 │   │   │   │   # multi-line %%time case                                                  │
│   1328 │   │   │   │   if expr_val is not None:                                                  │
│ in <module>:6                                                                                    │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/pandas/core/indexing.py:1067 in __getitem__              │
│                                                                                                  │
│   1064 │   │   │   key = tuple(com.apply_if_callable(x, self.obj) for x in key)                  │
│   1065 │   │   │   if self._is_scalar_access(key):                                               │
│   1066 │   │   │   │   return self.obj._get_value(*key, takeable=self._takeable)                 │
│ ❱ 1067 │   │   │   return self._getitem_tuple(key)                                               │
│   1068 │   │   else:                                                                             │
│   1069 │   │   │   # we by definition only have the 0th axis                                     │
│   1070 │   │   │   axis = self.axis or 0                                                         │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/pandas/core/indexing.py:1254 in _getitem_tuple           │
│                                                                                                  │
│   1251 │   │                                                                                     │
│   1252 │   │   # ugly hack for GH #836                                                           │
│   1253 │   │   if self._multi_take_opportunity(tup):                                             │
│ ❱ 1254 │   │   │   return self._multi_take(tup)                                                  │
│   1255 │   │                                                                                     │
│   1256 │   │   return self._getitem_tuple_same_dim(tup)                                          │
│   1257                                                                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/pandas/core/indexing.py:1205 in _multi_take              │
│                                                                                                  │
│   1202 │   │   values: same type as the object being indexed                                     │
│   1203 │   │   """                                                                               │
│   1204 │   │   # GH 836                                                                          │
│ ❱ 1205 │   │   d = {                                                                             │
│   1206 │   │   │   axis: self._get_listlike_indexer(key, axis)                                   │
│   1207 │   │   │   for (key, axis) in zip(tup, self.obj._AX